In [1]:
'''
Rice grain quality recognition project

tutorial: https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/

used labelme to label rice: https://github.com/wkentaro/labelme
dependencies:
matterport/Mask_RCNN: https://github.com/matterport/Mask_RCNN#installation
mask_rcnn doesn't work with tensorflow 2.0, using 1.14 for this project

setup venv:
python3 -m venv riceEnv

problems:
annotation points are in decimal places, don't know why, for time being going to cut off decimals.


define folder heiarchy as:
training/
- train_labels.csv
- imgs/
    - full.jpg
    - bad.jpg
- annots/
    - full.json
    - bad.json

'''

"\nRice grain quality recognition project\n\ntutorial: https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/\n\nused labelme to label rice: https://github.com/wkentaro/labelme\ndependencies:\nmatterport/Mask_RCNN: https://github.com/matterport/Mask_RCNN#installation\n\nsetup venv:\npython3 -m venv riceEnv\n\nproblems:\nannotation points are in decimal places, don't know why, for time being going to cut off decimals.\n\n\ndefine folder heiarchy as:\ntraining/\n- train_labels.csv\n- imgs/\n    - full.jpg\n    - bad.jpg\n- annots/\n    - full.json\n    - bad.json\n\n"

In [2]:
import cv2
import os
import re
import json
import pandas as pd
import numpy as np
from mrcnn.utils import Dataset

from matplotlib import pyplot as plt
%matplotlib inline

train_dir = os.getcwd() + '/training/'

/Users/kyy/riceCV/envrice/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kyy/riceCV/envrice/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kyy/riceCV/envrice/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kyy/riceCV/envrice/lib/python3.7/site-packages/tensorflow/python/f

In [3]:
def extract_polygon(filename):
    
    with open(filename) as f:
        ann = json.load(f)
        
    height = ann['imageHeight']
    width = ann['imageWidth']
    all_polygon = ann['shapes']
    
    data = [(shape['label'], 
            np.array(shape['points'], 
            dtype=np.int16)) for shape in all_polygon]
    return height, width, polygon

In [4]:
class riceDataset(Dataset):
    def load_dataset(self, dataset_dir, is_train=True):
        
        images_dir = dataset_dir + 'imgs/'
        annots_dir = dataset_dir + 'annots/'
        
        with open(dataset_dir + 'train_labels.csv') as tl:
            labels = re.split('[,\n]', tl.read()[:-1])
            print(labels)
        
        for i, label in zip(range(len(labels)), labels):
            self.add_class('dataset', i, label)
        
        for filename in os.listdir(images_dir):
            image_id = filename[:-4]
            image_path = images_dir + filename
            annots_path = annots_dir + image_id + '.json'
            width, height, polygons = extract_polygon(annots_path)
            self.add_image('dataset', 
                           image_id=image_id, 
                           path=image_path,
                           width=width,
                           height=height,
                           polygons=polygons,
                           annotation=annots_path)
        
    def load_mask(self, image_id):
        
        info = self.image_info[image_id]
        path = info['annotation']
        h, w, boxes = self.extract_boxes(path)
        masks = np.zeros([h, w, len(boxes)], dtype='uint8')
        class_ids = []
        for i in range(len(boxes)):
            box = boxes[i]
            
    
#     def image_reference(self, image_id):

In [5]:

rice_data = riceDataset()
h, w, data = extract_polygon('./training/annots/bad.json')
print(h, w, data)
rice_data.load_dataset(train_dir)

NameError: name 'extract_boxes' is not defined